In [61]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
def spx_process(filename):
    raw = pd.read_csv(filename, 
                      names = ["datetime", "open", "high", "low", "close"])
    raw.datetime = pd.to_datetime(raw.datetime, format = '%Y.%m.%d %H:%M:%S')
    raw["date"] = raw.datetime.dt.date
    raw["time"] = raw.datetime.dt.time
    if filename == "SPX_2014_12.csv":
        data = raw.loc[raw.datetime > "2013-01-01",:]
    elif filename == "SPX_2016_12.csv":
        data = raw.loc[raw.datetime > "2015-01-01",:]
    else:
        data = raw
    before_930 = pd.to_timedelta(data.time.astype(str)) < pd.to_timedelta('09:30:00')
    after_359 = pd.to_timedelta(data.time.astype(str)) > pd.to_timedelta('15:59:00')
    delete_index = before_930 | after_359
    data = data.loc[~delete_index,:]
    return data

In [27]:
data1 = spx_process("SPX_2014_12.csv")

In [28]:
data2 = spx_process("SPX_2016_12.csv")

In [29]:
data3 = spx_process("SPX_2018_12.csv")

In [30]:
data4 = spx_process("SPX_2019_12.csv")

In [31]:
data = pd.concat([data1, data2, data3, data4], axis=0)

In [32]:
data = data.reset_index().drop(columns = ["index", "time", "date"])

In [33]:
data.to_csv('all_cleaned_SPX_data.csv', index=False)

In [34]:
data_every_five_min = data.loc[range(0, data.shape[0], 5), "close"]

In [53]:
five_min_return = (data_every_five_min.diff()/data_every_five_min)[1:]

In [58]:
five_min_up_or_down = pd.Series(five_min_return).map(lambda x: "down" if x < 0 else "up")

In [69]:
five_min_up_or_down.value_counts(normalize=True)

up      0.512593
down    0.487407
Name: close, dtype: float64